In [ ]:
################################################
# New style 
################################################
import sys
import os

workdir_ = '/glade/work/juliob'
if ( workdir_ not in sys.path ):
    sys.path.append(workdir_)
    print( f" a path to {workdir_} added in {__name__} ")

from myPythonTools.Utils import utils as uti
from myPythonTools.Utils import numerical_utils as nuti
from myPythonTools.Utils import AveragingUtils as Av
from myPythonTools.Utils import validation_data as Val
from myPythonTools.Utils import PlotUtil as Pu
from myPythonTools.Plotting import LatLonMaps as LL 
from myPythonTools.CASutils import filter_utils as fu 

from PyRegridding.Utils import GridUtils as GrU
from PyRegridding.Utils import MakePressures as MkP
from PyRegridding.Drivers import RegridField as RgF


# The usual
from datetime import date
import numpy as np
import xarray as xr
import matplotlib.pyplot as plt
import matplotlib.tri as tri

# Cartopy for pretty maps
import cartopy
import cartopy.crs as ccrs
import cartopy.feature as cfeature

# Some other useful packages 
import importlib
import copy
import time
import cftime

importlib.reload( uti )
importlib.reload( Pu )
importlib.reload(Av)
importlib.reload(Val)
#importlib.reload(vAB)
importlib.reload(MkP)
importlib.reload(RgF)


############################
# Initialize dict for regridding
############################
RgObs={}


In [ ]:
Src = 'ERA5'
SrcInfo = GrU.gridInfo(Src) #,Vgrid=DstVgrid)

Dst = 'fv1x1'
DstInfo = GrU.gridInfo(Dst) #,Vgrid=DstVgrid)
lat_x,lon_x = GrU.latlon( scrip= DstInfo['scrip'], Hkey=DstInfo['Hkey'] )

RegridObj = GrU.regrid_object_lib(RgOb=RgObs, src=Src, dst=Dst, RegridMethod='CONSERVE_2ND')

In [ ]:
lev_a=A.X.lev.values
zlev_a=-7.0*np.log( lev_a/1_000. )

AAa_s_yx=RgF.Horz(xfld_Src=AAa_s , Src=Src, Dst=Dst , RegridObj_In= RegridObj )
A['AA_s_yx'] = AAa_s_yx

In [ ]:
SuperDir = "/glade/campaign/cgd/amp/juliob/ERA5"

#for year in np.arange( start=1994, stop=1995 ):
#    for month in np.arange( start=1, stop=13 ):
for year in np.arange( start=1991, stop=1995 ):
    for month in np.arange( start=1, stop=13 ):
        
        fixed_monthly_dir = f"{SuperDir}/Fluxes/VQ/monthly/fixed-{str(year).zfill(4)}/"
        fixed_monthly_fil = f"{fixed_monthly_dir}/e5.oper.an.ml.VQ.{str(year).zfill(4)}-{str(month).zfill(2)}.nc"
        regrd_monthly_dir = f"{SuperDir}/Fluxes/VQ/monthly/{Dst}-{str(year).zfill(4)}/"
        regrd_monthly_fil = f"{regrd_monthly_dir}/e5.oper.an.ml.VQ.{Dst}.{str(year).zfill(4)}-{str(month).zfill(2)}.nc"
        os.makedirs( regrd_monthly_dir , exist_ok=True )
        
        
        print(f"Open {fixed_monthly_fil}" )
        Mly = xr.open_dataset( fixed_monthly_fil )  
        Mly = Mly.drop_vars('DateStamp',errors='ignore')
        Mly = Mly.drop_vars('times_in_avg',errors='ignore')
        Mly = Mly.drop_dims('days', errors='ignore')

        print( list(Mly.data_vars) )

        vars = list(Mly.data_vars)
        coords = dict( 
            longitude  = ( ["longitude"],lon_x ),
            latitude   = ( ["latitude"],lat_x ),
            lev  = ( ["lev"],Mly.lev.values),
            ilev = ( ["ilev"],Mly.ilev.values ),
            time = ( ["time"],Mly.time.values ), )
            #time = ( ["time"],np.datetime64(f"{str(year).zfill(4)}-{str(month).zfill(2)}-15T00:00:00","ns") ), ) 
            
        Mly_rg = xr.Dataset( coords=coords  )

        for var in vars:
            print( f'{var} ..: Dims - {Mly[var].dims} ')
            print( f'{var} ..: Attr - {Mly[var].attrs} ')

            aa   = Mly[var].values        
            if ('longitude' in Mly[var].dims) and ('latitude' in Mly[var].dims):
                aa_x = RgF.Horz(xfld_Src=aa , Src=Src, Dst=Dst , RegridObj_In= RegridObj ) 
            else:
                aa_x = aa

            if ( var == 'a_half'):
                attrs=dict( description='interface hybrid eta coordinate A-coeff ',units='Pa',) 
            elif ( var == 'b_half'):
                attrs=dict( description='interface hybrid eta coordinate B-coeff ',units='1',) 
            elif ( var == 'a_model'):
                attrs=dict( description='mid-level hybrid eta coordinate A-coeff ',units='Pa',) 
            elif ( var == 'b_model'):
                attrs=dict( description='mid-level hybrid eta coordinate B-coeff ',units='1',) 
            elif ( var == 'SP'):
                attrs=dict( description='Surface Pressure',units='Pa',) 
            elif ( var == 'DELP'):
                attrs=dict( description='layer Pressure thickness',units='Pa',) 
            elif ( var == 'VQ'):
                attrs=dict( description='meridional q-flux',units='(m s-1)(kg kg-1)',) 
            elif ( var == 'dPVQ'):
                attrs=dict( description='p-thickness weighted meridional q-flux',units='Pa (m s-1)(kg kg-1)',) 
            else:
                attrs=None
        
            Dar = xr.DataArray( data=aa_x , dims=(Mly[var].dims),
                                attrs=attrs ,) 

            Mly_rg[var] = Dar

        print( f'write to {regrd_monthly_fil} ')
        Mly_rg.to_netcdf( regrd_monthly_fil ) 

        
        """
        
        
        flds=['SP','VQ','DELP','dPVQ','DateStamp','times_in_avg']
        for fld in flds:
            Mly[fld] = Mly[fld].expand_dims(time=[np.datetime64(f"{str(year).zfill(4)}-{str(month).zfill(2)}-15T00:00:00","ns")])
        
        Mly.to_netcdf( fixed_monthly_fil )
        print( f"    .. Wrote {fixed_monthly_fil}")
        """

In [ ]:
Mly_rg.dPVQ

In [ ]:

dpvq_m =Mly.dPVQ.values


In [ ]:
dpvq_d_m = np.average( dpvq_d , axis=0)

In [ ]:
err= dpvq_m.flatten() - dpvq_d_m.flatten()

In [ ]:
a=dpvq_m.flatten()
print( np.sqrt(np.mean((a - np.mean(a))**2)) )


In [ ]:
print( np.sqrt( np.average( err**2 )) )

In [ ]:

plt.plot( err )
